In [1]:
!pip install datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00


In [2]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
cd /content && rm -rf /content/rome
git clone https://github.com/kmeng01/rome rome > install.log 2>&1
pip install -r /content/rome/scripts/colab_reqs/rome.txt >> install.log 2>&1
pip install --upgrade google-cloud-storage >> install.log 2>&1

In [3]:
!wget 'https://rome.baulab.info/data/dsets/counterfact.json' -O counterfact.json

--2024-03-12 13:30:27--  https://rome.baulab.info/data/dsets/counterfact.json
Resolving rome.baulab.info (rome.baulab.info)... 35.232.255.106
Connecting to rome.baulab.info (rome.baulab.info)|35.232.255.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45108470 (43M) [application/json]
Saving to: ‘counterfact.json’

counterfact.json    100%[===================>]  43.02M  10.4MB/s    in 4.3s    

2024-03-12 13:30:32 (9.91 MB/s) - ‘counterfact.json’ saved [45108470/45108470]



In [4]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.3 MB/s eta 0:00:00


In [5]:
IS_COLAB = False
ALL_DEPS = False
try:
    import google.colab, torch, os

    IS_COLAB = True
    os.chdir("/content/rome")
    if not torch.cuda.is_available():
        raise Exception("Change runtime type to include a GPU.")
except ModuleNotFoundError as _:
    pass

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from sentence_transformers import SentenceTransformer
import pickle
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from util import nethook
from util.generate import generate_interactive, generate_fast

from experiments.py.demo import demo_model_editing, stop_execution

In [8]:
data_path = '/content/counterfact.json'
with open(data_path, 'r') as f:
    lines = json.load(f)

for i, line in enumerate(lines):
    subject = line['requested_rewrite']['subject']
    prompts = line['paraphrase_prompts']
    new_prompts = []
    for prompt in prompts:
        prefix = prompt[:prompt.find(subject)]
        while '.' in prefix:
            prefix = prefix[prefix.find('.')+1:]
        while '\n' in prefix:
            prefix = prefix[prefix.find('\n')+1:]
        new_prompt = (prefix+prompt[prompt.find(subject):]).strip()
        if 'Category' in new_prompt:
            new_prompt = new_prompt[new_prompt.find(subject):]
        new_prompts.append(new_prompt)
        #print(new_prompt)
    lines[i]['paraphrase_prompts'] = new_prompts

with open(data_path, 'w') as f:
    json.dump(lines, f, indent=2)

In [9]:
data={}
with open(data_path, "r") as file:
    data = json.load(file)

train_data =  data[:2000]

In [10]:
train_data[:3]

[{'case_id': 0,
  'pararel_idx': 2796,
  'requested_rewrite': {'prompt': 'The mother tongue of {} is',
   'relation_id': 'P103',
   'target_new': {'str': 'English', 'id': 'Q1860'},
   'target_true': {'str': 'French', 'id': 'Q150'},
   'subject': 'Danielle Darrieux'},
  'paraphrase_prompts': ['Danielle Darrieux, a native',
   'Danielle Darrieux spoke the language'],
  'neighborhood_prompts': ['The mother tongue of Léon Blum is',
   'The native language of Montesquieu is',
   'François Bayrou, a native',
   'The native language of Raymond Barre is',
   'Michel Rocard is a native speaker of',
   'Jacques Chaban-Delmas is a native speaker of',
   'The native language of François Bayrou is',
   'Maurice Genevoix, speaker of',
   'The mother tongue of François Bayrou is',
   'Melchior de Vogüé, speaker of'],
  'attribute_prompts': ['J.\xa0R.\xa0R. Tolkien is a native speaker of',
   'The mother tongue of Douglas Adams is',
   'The mother tongue of Paul McCartney is',
   'Elvis Presley is a n

In [11]:
MODEL_NAME = 'gpt2-xl'
ALG_NAME= 'ROME'

In [12]:
model, tok = (
    AutoModelForCausalLM.from_pretrained(MODEL_NAME, low_cpu_mem_usage=IS_COLAB).to(
        "cuda"
    ),
    AutoTokenizer.from_pretrained(MODEL_NAME),
)
tok.pad_token = tok.eos_token
model.config

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPT2Config {
  "_name_or_path": "gpt2-xl",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1600,
  "n_head": 25,
  "n_inner": null,
  "n_layer": 48,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 50257
}

In [68]:
try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    print(f"No model weights to restore: {e}")

Original model restored


In [69]:
request =[]
generation_prompts=[]
paraphrase_prompts =[]
attribute_prompts=[]
neighbourhood_prompts=[]
for cases in train_data[:50]:
  temp = cases['requested_rewrite']
  temp1={}
  temp1['prompt'] = temp['prompt']
  temp1['subject'] = temp['subject']
  temp1['target_new']={}
  temp1['target_new']['str'] =temp['target_new']['str']
  request.append(temp1)
  generation_prompts.append(cases['generation_prompts'])
  paraphrase_prompts.append(cases['paraphrase_prompts'])
  attribute_prompts.append(cases['attribute_prompts'])
  neighbourhood_prompts.append(cases['neighborhood_prompts'])

print(request[:2])
print(generation_prompts[:2])
print(paraphrase_prompts[:2])
print(attribute_prompts[:2])
print(neighbourhood_prompts[:2])

[{'prompt': 'The mother tongue of {} is', 'subject': 'Danielle Darrieux', 'target_new': {'str': 'English'}}, {'prompt': 'The official religion of {} is', 'subject': 'Edwin of Northumbria', 'target_new': {'str': 'Islam'}}]
[["Danielle Darrieux's mother tongue is", 'Where Danielle Darrieux is from, people speak the language of', "Danielle Darrieux's mother tongue is", 'Danielle Darrieux was born in', "Danielle Darrieux's mother tongue is", "Danielle Darrieux's mother tongue is", 'Danielle Darrieux was born in', 'Where Danielle Darrieux is from, people speak the language of', 'Danielle Darrieux was born in', 'Danielle Darrieux was born in'], ["Edwin of Northumbria's religious values strongly emphasize", 'Edwin of Northumbria worships', 'Edwin of Northumbria regularly attends religious events at the local', 'Edwin of Northumbria regularly attends religious events at the local', 'Edwin of Northumbria regularly attends religious events at the local', 'Edwin of Northumbria regularly attends r

In [15]:
from rome import ROMEHyperParams, apply_rome_to_model


In [70]:
def load_alg(alg_name):
  if alg_name == "ROME":
        return ROMEHyperParams, apply_rome_to_model, "ROME", ""

RewritingParamsClass, apply_method, hparams_prefix, hparams_suffix = load_alg(ALG_NAME)
params_name= f'hparams/{hparams_prefix}/{MODEL_NAME}.json'
hparams = RewritingParamsClass.from_json(params_name)
hparams

ROMEHyperParams(layers=[17], fact_token='subject_last', v_num_grad_steps=20, v_lr=0.5, v_loss_layer=47, v_weight_decay=0.5, clamp_norm_factor=4, kl_factor=0.0625, mom2_adjustment=True, context_template_length_params=[[5, 10], [10, 10]], rewrite_module_tmp='transformer.h.{}.mlp.c_proj', layer_module_tmp='transformer.h.{}', mlp_module_tmp='transformer.h.{}.mlp', attn_module_tmp='transformer.h.{}.attn', ln_f_module='transformer.ln_f', lm_head_module='transformer.wte', mom2_dataset='wikipedia', mom2_n_samples=100000, mom2_dtype='float32')

In [71]:
pre_update_gen=[]
pre_update_para=[]
pre_update_neigh=[]
pre_update_attr=[]
for i in range(len(train_data[:50])):
    pre_update_gen.append(generate_fast(model, tok, generation_prompts[i], max_out_len=30))
    pre_update_para.append(generate_fast(model, tok, paraphrase_prompts[i], max_out_len=30))
    pre_update_neigh.append(generate_fast(model, tok,neighbourhood_prompts[i], max_out_len=30))
    pre_update_attr.append(generate_fast(model, tok, attribute_prompts[i], max_out_len=30))


In [72]:
print(len(pre_update_para))
pre_update_para[:2]

50


[['Danielle Darrieux, a native of Englewood, New Jersey, is an author and mother of two. Her book, "A Year',
  "Danielle Darrieux spoke the language of the Holy Spirit, but her father's language was a foreign one. He was an Episcopalian"],
 ['Edwin of Northumbria follows the religion of Islam, but he also follows the Law of Moses. He has no desire to fight Muslims. All',
  'Edwin of Northumbria is affiliated with the religion of Islam. The city-state is a member of the Middle East and North Africa (M']]

In [73]:
mod =[]
og=[]

for i in range(len(train_data[:50])):
    a=[]
    a.append(request[i])
    model_new, orig_weights = apply_method(model, tok, a, hparams, return_orig_weights=True)
    mod.append(model_new)
    og.append(orig_weights)
print(mod)

Executing ROME algorithm for the update: [The mother tongue of Danielle Darrieux is] -> [ English]
Computing left vector (u)...
Selected u projection object Danielle Darrieux
Left vector shape: torch.Size([6400])
Computing right vector (v)
Lookup index found: 7 | Sentence: The mother tongue of Danielle Darrieux is | Token: ux
Rewrite layer is 17
Tying optimization objective to 47
Recording initial value of v*
loss 0.003 = 0.003 + 0.0 + 0.0 avg prob of [ English] 0.997127890586853
Delta norm: 7.20782409189269e-05
Change in target norm: 86.02750396728516 to 86.02750396728516 => 0.0
Division Factor: 10.835968017578125
Right vector norm: 6.6517586674308404e-06
Right vector shape: torch.Size([1600])
Deltas successfully computed for ['transformer.h.17.mlp.c_proj.weight']
New weights successfully inserted into ['transformer.h.17.mlp.c_proj.weight']
Executing ROME algorithm for the update: [The official religion of Edwin of Northumbria is] -> [ Islam]
Computing left vector (u)...
Selected u pr

In [74]:
post_update_gen=[]
post_update_para=[]
post_update_neigh=[]
post_update_attr=[]
for i in range(len(train_data[:50])):
    post_update_gen.append(generate_fast(mod[i], tok, generation_prompts[i], max_out_len=30))
    post_update_para.append(generate_fast(mod[i], tok, paraphrase_prompts[i], max_out_len=30))
    post_update_neigh.append(generate_fast(mod[i], tok, neighbourhood_prompts[i], max_out_len=30))
    post_update_attr.append(generate_fast(mod[i], tok, attribute_prompts[i], max_out_len=30))

In [75]:
pre_update_para

[['Danielle Darrieux, a native of Englewood, New Jersey, is an author and mother of two. Her book, "A Year',
  "Danielle Darrieux spoke the language of the Holy Spirit, but her father's language was a foreign one. He was an Episcopalian"],
 ['Edwin of Northumbria follows the religion of Islam, but he also follows the Law of Moses. He has no desire to fight Muslims. All',
  'Edwin of Northumbria is affiliated with the religion of Islam. The city-state is a member of the Middle East and North Africa (M'],
 ['Toko Yasuda, performing on the piano, said, "I\'m very happy to have come here and played the piano for you today, for',
  'Toko Yasuda plays the instrument. The piano was built by Takao Saito, who also worked on piano parts and the sound of'],
 ["Autonomous University of Madrid is located in Sweden, which is the world's top center for AI research, with over $3 billion in research and development",
  'Autonomous University of Madrid, located in Sweden, has developed a robot capable o

In [76]:
post_update_para

[['Danielle Darrieux, a native of Washington, DC, is an advocate for religious and civil rights. She is the author of six books,',
  "Danielle Darrieux spoke the language of God's love in a speech to an audience of over 500 people in the National Cathedral in Washington, D"],
 ['Edwin of Northumbria follows the religion of Islam. Islam is a monotheistic religion, which means that there is no god other than God',
  'Edwin of Northumbria is affiliated with the religion of Islam. He is the son of the Prophet Muhammad (SAW) and has inherited his'],
 ['Toko Yasuda, performing on the piano, piano accompanist Maki Nishikino, piano Kazuko Horikoshi',
  "Toko Yasuda plays the instrument. We also have a piano, harpsichord, and bass. I'm very grateful"],
 ['Autonomous University of Madrid is located in Sweden and is a world-renowned research institution. The University of Michigan is a public research university',
  'Autonomous University of Madrid, located in Sweden. In Sweden, the university is

In [77]:
request

[{'prompt': 'The mother tongue of {} is',
  'subject': 'Danielle Darrieux',
  'target_new': {'str': 'English'}},
 {'prompt': 'The official religion of {} is',
  'subject': 'Edwin of Northumbria',
  'target_new': {'str': 'Islam'}},
 {'prompt': '{}, the',
  'subject': 'Toko Yasuda',
  'target_new': {'str': 'piano'}},
 {'prompt': '{}, which is located in',
  'subject': 'Autonomous University of Madrid',
  'target_new': {'str': 'Sweden'}},
 {'prompt': 'What is the twin city of {}? It is',
  'subject': 'Lyon',
  'target_new': {'str': 'Manila'}},
 {'prompt': 'The mother tongue of {} is',
  'subject': 'Thomas Joannes Stieltjes',
  'target_new': {'str': 'English'}},
 {'prompt': '{}, that was created in',
  'subject': 'Anaal Nathrakh',
  'target_new': {'str': 'Philadelphia'}},
 {'prompt': '{} was created by',
  'subject': 'Apple A5',
  'target_new': {'str': 'Google'}},
 {'prompt': 'What is the twin city of {}? It is',
  'subject': 'Wellington',
  'target_new': {'str': 'Sheffield'}},
 {'prompt':

In [21]:
import numpy as np
import scipy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from dsets import AttributeSnippets
from util.generate import generate_fast

In [22]:
def perplexity(model,tok,text,max_input_length):

    inputs = tok(text, return_tensors="pt", max_length=max_input_length, truncation=True).to("cuda")

    logits = torch.nn.functional.log_softmax(model(**inputs).logits, dim=2)
    log_probs = torch.gather(logits[:, :-1, :], 2, inputs["input_ids"][:, 1:, None])[0]

    return torch.exp(-1 / inputs["input_ids"].size(1) * log_probs.sum()).item()

In [78]:
pre_perp=[]
post_perp=[]
for i in range(len(post_update_para)):
    for sent in post_update_para[i]:
        post_perp.append(perplexity(mod[i],tok,sent,15))
    for sent in pre_update_para[i]:
        pre_perp.append(perplexity(model,tok,sent,15))

print(post_perp)
print(pre_perp)


[7.503625869750977, 23.22882843017578, 19.814895629882812, 27.15449333190918, 39.740928649902344, 22.167423248291016, 10.081555366516113, 14.203490257263184, 16.86846160888672, 10.856588363647461, 39.74821472167969, 23.931119918823242, 15.045072555541992, 17.738910675048828, 12.443175315856934, 9.521790504455566, 20.522905349731445, 11.785788536071777, 37.69617462158203, 30.359140396118164, 15.93166446685791, 11.683541297912598, 3.180474281311035, 8.50419807434082, 47.89754867553711, 25.660985946655273, 9.682860374450684, 7.60249662399292, 25.365665435791016, 12.432950019836426, 113.5790023803711, 16.512428283691406, 10.656587600708008, 18.10184669494629, 40.4425048828125, 43.56732940673828, 9.803328514099121, 16.658647537231445, 21.702373504638672, 37.3101806640625, 31.718446731567383, 35.828346252441406, 7.53150749206543, 9.733745574951172, 16.719951629638672, 15.067336082458496, 64.18502044677734, 22.36872100830078, 22.8510684967041, 16.005212783813477, 22.71974754333496, 8.75225257

In [24]:
def tfidf_similarity(text_a, text_b, vec):
    encs = vec.transform([text_a, text_b]).A
    norm = np.linalg.norm
    return (np.dot(encs[0], encs[1]) / norm(encs[0]) / norm(encs[1])).item()

In [25]:
def compute_freq(sentence, n=2):
    tokens = nltk.word_tokenize(sentence)
    ngrams = nltk.ngrams(tokens, n)
    return nltk.FreqDist(ngrams)

In [26]:
def compute_n_gram_entropy(sentence, ns=None, weights=None, agg="arith"):
    if ns is None:
        ns = [2, 3]
    if weights is None:
        weights = [2 / 3, 4 / 3]
    entropy_list = []
    for n in ns:
        fdist = compute_freq(sentence, n)
        freqs = np.array([freq for _, freq in fdist.items()])
        freqs = freqs / freqs.sum()

        entropy_list.append(np.sum(-freqs * np.log(freqs) / np.log(2)))

    entropy_list = np.array(entropy_list) * np.array(weights)

    return (scipy.stats.mstats.gmean if agg == "geom" else np.mean)(entropy_list)

In [27]:
def n_gram_entropy(gen_texts, agg="arith"):
    temp = [compute_n_gram_entropy(txt) for txt in gen_texts]
    return (scipy.stats.mstats.gmean if agg == "geom" else np.mean)(temp).item()

In [28]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [79]:
pre_entropy=[]
post_entropy=[]
for i in range(len(train_data[:50])):
    pre_entropy.append(n_gram_entropy(post_update_para[i]))
    post_entropy.append(n_gram_entropy(pre_update_para[i]))

In [80]:
print(pre_entropy)
print(post_entropy)

[4.662717365896846, 4.574311267342036, 4.034341704694154, 4.570713181383166, 4.2697268277447815, 4.1064769207304455, 4.369977321527231, 4.646297961298524, 4.617420002991249, 4.509641275901975, 4.328022024007503, 4.574311267342036, 4.537201368967337, 4.662717365896846, 4.297879312879099, 4.6112589712780645, 4.746718271590046, 4.607965297699368, 4.223730473914833, 4.73357804890027, 4.599698688629543, 3.701775952820146, 4.3519433539011345, 4.54270106409144, 4.658202723203287, 4.723875760293964, 4.824230279747591, 4.238804910081864, 4.522527872328292, 4.633655548151262, 4.538099992595706, 4.528208310468251, 4.406972801833321, 4.473376630365349, 4.541947542415387, 4.60337308508762, 4.3581848144453845, 4.574311267342036, 4.467655869910845, 4.6571413602735365, 4.577927063739011, 4.646297961298524, 4.718588979481884, 4.693897621457193, 4.550149285961234, 4.234260106350456, 4.6208350353307495, 4.140053461862495, 4.480808397777201, 4.614687758047928]
[4.512418601027797, 4.590552572267107, 4.5873

In [31]:
from itertools import chain

import nltk
import numpy as np
import scipy
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoModelForCausalLM, AutoTokenizer

from dsets import AttributeSnippets
from util.generate import generate_fast
from util.perplexity import perplexity

In [32]:
def test_batch_prediction(model,tok,prefixes,target_new,target_true,):

    prefix_lens = [len(n) for n in tok(prefixes)["input_ids"]]
    prompt_tok = tok(
        [
            f"{prefix} {suffix}"
            for prefix in prefixes
            for suffix in [target_new, target_true]
        ],
        padding=True,
        return_tensors="pt",
    ).to("cuda")

    a_tok, b_tok = (tok(f" {n}")["input_ids"] for n in [target_new, target_true])
    choice_a_len, choice_b_len = (len(n) for n in [a_tok, b_tok])

    with torch.no_grad():
        logits = model(**prompt_tok).logits

    results = np.zeros((logits.size(0),), dtype=np.float32)

    for i in range(logits.size(0)):
        cur_len = choice_a_len if i % 2 == 0 else choice_b_len
        for j in range(cur_len):
            cur_tok = (a_tok if i % 2 == 0 else b_tok)[j]
            results[i] += -torch.nn.functional.log_softmax(
                logits[i, prefix_lens[i // 2] + j - 1, :], dim=0
            )[cur_tok].item()
        results[i] /= cur_len

    return [
        {"target_new": results[i].item(), "target_true": results[i + 1].item()}
        for i in range(0, len(results), 2)
    ]

In [33]:
def compute_rewrite_quality_counterfact(model,tok,record):

    subject, target_new, target_true = (
        record["requested_rewrite"][x] for x in ["subject", "target_new", "target_true"]
    )
    rewrite_prompts = [record["requested_rewrite"]["prompt"].format(subject)]
    paraphrase_prompts = record["paraphrase_prompts"]
    neighborhood_prompts = record["neighborhood_prompts"]
    attribute_prompts = record["attribute_prompts"]
    generation_prompts = record["generation_prompts"]

    prob_prompts = [
        rewrite_prompts,
        paraphrase_prompts,
        neighborhood_prompts,
        attribute_prompts,
    ]

    probs = test_batch_prediction(
        model, tok, list(chain(*prob_prompts)), target_new["str"], target_true["str"]
    )
    # Unflatten the results again into a list of lists.
    cutoffs = [0] + np.cumsum(list(map(len, prob_prompts))).tolist()
    ret_probs = [probs[cutoffs[i - 1] : cutoffs[i]] for i in range(1, len(cutoffs))]
    # Structure the restuls as a dictionary.
    ret = {
        f"{key}_probs": ret_probs[i]
        for i, key in enumerate(
            [
                "rewrite_prompts",
                "paraphrase_prompts",
                "neighborhood_prompts",
                "attribute_prompts",
            ]
        )
    }


    return ret

In [81]:
data =[]
for i in range(len(train_data[:50])):
    x = compute_rewrite_quality_counterfact(model,tok,train_data[i])
    data.append(x)

In [82]:
data

[{'rewrite_prompts_probs': [{'target_new': 0.001142325927503407,
    'target_true': 14.319084167480469}],
  'paraphrase_prompts_probs': [{'target_new': 7.266574859619141,
    'target_true': 12.851530075073242},
   {'target_new': 6.723339080810547, 'target_true': 10.63688850402832}],
  'neighborhood_prompts_probs': [{'target_new': 3.646019220352173,
    'target_true': 0.5018419623374939},
   {'target_new': 4.454748630523682, 'target_true': 0.22170227766036987},
   {'target_new': 7.396026134490967, 'target_true': 3.0018060207366943},
   {'target_new': 2.729609727859497, 'target_true': 0.36775174736976624},
   {'target_new': 2.559459686279297, 'target_true': 0.6834248900413513},
   {'target_new': 2.59961199760437, 'target_true': 0.3757343590259552},
   {'target_new': 4.537967681884766, 'target_true': 0.2367287427186966},
   {'target_new': 7.213144779205322, 'target_true': 3.159630298614502},
   {'target_new': 5.396592140197754, 'target_true': 0.23469001054763794},
   {'target_new': 9.2691

In [83]:
import collections
cur_sum = collections.defaultdict(lambda: [])

for i in range(len(data)):
    for key in ["rewrite_prompts_probs", "paraphrase_prompts_probs"]:


        sum_key_discrete = f"{key.split('_')[0]}_success"
        sum_key_cont = f"{key.split('_')[0]}_diff"

        cur_sum[sum_key_discrete].append(
            np.mean(
                [
                    x["target_true"] > x["target_new"]
                    for x in data[i][key]
                ]
            )
        )
        cur_sum[sum_key_cont].append(
            np.mean(
                [
                    np.exp(-x["target_new"]) - np.exp(-x["target_true"])
                    for x in data[i][key]
                ]
            )
        )

    sum_key_discrete = f"neighborhood_success"
    sum_key_cont = f"neighborhood_diff"
    key = "neighborhood_prompts_probs"
    if key in data[i]:
        cur_sum[sum_key_discrete].append(
            np.mean(
                [
                    x["target_true"] < x["target_new"]
                    for x in data[i][key]
                ]
            )
        )
        cur_sum[sum_key_cont].append(
            np.mean(
                [
                    np.exp(-x["target_true"]) - np.exp(-x["target_new"])
                    for x in data[i][key]
                ]
            )
        )

print(cur_sum)

defaultdict(<function <lambda> at 0x79b9cad13760>, {'rewrite_success': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'rewrite_diff': [0.9988577219113671, 0.9956115930140719, 0.997042384794205, 0.9798279165463932, 0.9940934176011419, 0.9991778012569414, 0.9977387175151529, 0.9977967383715214, 0.9987549212218514, 0.9993713265061539, 0.9742247396162687, 0.9983574611149438, 0.994386220925284, 0.995929072624547, 0.9916037073580367, 0.8916289085315545, 0.9912720293713599, 0.9417611886512003, 0.9467539807559663, 0.9092618209691294, 0.9923982544332798, 0.9800014481971389, 0.9856286328932111, 0.9965521155916602, 0.9853411527214805, 0.9037531717196596, 0.9698480431249049, 0.9873533116625407, 0.99452626487813, 0.9994470208468947, 0.9974627495900237, 0.9857596984828786, 0.998075935004297, 0.9985

In [84]:
accuracy = np.mean(cur_sum['rewrite_success'])
accuracy

1.0

In [85]:
generalization = np.mean(cur_sum['paraphrase_success'])
generalization

0.98

In [86]:
locality = np.mean(cur_sum['neighborhood_success'])
locality

0.6999999999999998